전이학습
```
Fine-tuning
  모델의 파라메터를 새로운 테스크의 데이터로 추가 학습해서 최적화 과정  
  방법 : 모델의 모든레이어(일부 레이어) 업데이트
    테스크별 출력 레이어(분류를 위한 linear layer)를 추가 학습
  단점 : 자원소비가 많죠, 데이터가 적으면 과적합
  활용 : 텍스트분류, 질문응답(QA),개체명인식(NER)
Feature-Extraction
  사전 학습된 모델을 고정된 특성 추출기로 사용, 모델의 출력을 활용해 새로운 분류기(MLP, SVM) 학습
  방법 : 모델의 가중치를 고정(freeze)하고 출력의 토큰 임베딩만 추출
  추출된 임베딩을 새로운 머신러닝 모델 학습
  단점 : 성능이 fine tuning에 비해 떨어짐
  활용 : 빠르게 확인, 데이터가 매우 적은
전략 : 초기에는 Feature-Extraction 으로 테스트 성능이 부족하면 Fine-tuning으로 전환

```

NLP(자연어)에서 전이학습이 필요한 이유
```
데이터 부족문제
언어의 복잡성
사례
  BERT는 wikipedia와 BookCorpus로 사전학습 문맥이해 강력
  ->Fine-Tuning해서 SQuAD(질문응답),
  한국어 KoBERT, KLUE-BERT 한국어 데이터로 사전학습  
한국어 특화모델
  koBERT : SKT에서 한국어 데이터로 학습
  KLUE-BERT : 한국어 벤치마크(KLUE)학습
  KoGPT2  : SKT에서 한국에 데이터로 학습
```

전이학습을 위한 텍스트 전처리 및 텍스트 표현방법
```
데이터전처리
  토큰화
    텍스트를 단어, 서브워드, 문자 단위로 분류
  패딩
    입력 길이 고정
  마스킹
    Attention Mask : 패딩 토큰을 Attention 계산에서 제외
    1 1 1 1 0 0
  정규화
    소문자, 특수문자제거, 형태소 분석
  도메인 특화 전처리
    의료(법률)데이터 : 전문용어 사전 추가
텍스트표현
  임베딩
  positional Encoding : 위치 정보(헤드 단위로 병렬처리)
전처리 시 고려 사항
  입력길이 : 모델 마다 최대 토큰수 제한(BERT 512, gpt-3:2048)
  다국어 처리 : 한국어 영어 혼합 모델 (bert-base-multi...)
  데이터 증강 : 역변역이나 동의어 치환 활용
```

허깅 페이스의 파이프라인을 활욯해서 전이 학습없이 감정 분류를 수행하고 전이학습의 기본 흐름을 파악

In [ ]:
from transformers import pipeline
model_name = 'bert-base-multilingual-cased'
classifier = pipeline('sentiment-analysis', model=model_name, tokenizer=model_name)
classifier

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# 한국어와 영어 혼합
texts = [
    '이 영화 정말 재미있어요',
    'This movie is abosoluterly terrible',
    '배우들의 연기가 훌륭했어요',
    "I was bored througtout"
]
# 추론
results = classifier(texts)
for result in results:
  print(result)

{'label': 'LABEL_0', 'score': 0.5663211345672607}
{'label': 'LABEL_0', 'score': 0.5256982445716858}
{'label': 'LABEL_0', 'score': 0.5583690404891968}
{'label': 'LABEL_0', 'score': 0.5231467485427856}


In [ ]:
# 다국어 베이스 - 파인튜닝이 필요한 모델(특정 상황메 맞게 튜닝된 모델이 아님)
from transformers import pipeline
model_name = 'google-bert/bert-base-multilingual-uncased'
classifier = pipeline('sentiment-analysis', model=model_name, tokenizer=model_name)
# 한국어와 영어 혼합
texts = [
    "이 영화 정말 재미있어요",
    "This movie is absolutely terrible",
    "배우들의 연기가 훌륭했어요",
    "I was bored througtout the film"
]
# 추론
results = classifier(texts)
for result in results:
    print(result)

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/672M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

Device set to use cpu


{'label': 'LABEL_0', 'score': 0.5538701415061951}
{'label': 'LABEL_0', 'score': 0.5457417964935303}
{'label': 'LABEL_0', 'score': 0.54917311668396}
{'label': 'LABEL_0', 'score': 0.5411763787269592}


In [ ]:
# 5점 척도(1~5점) 감성분석 모델
from transformers import pipeline
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'
classifier = pipeline('sentiment-analysis', model=model_name, tokenizer=model_name)
# 한국어와 영어 혼합
texts = [
    "이 영화 정말 재미있어요",
    "This movie is absolutely terrible",
    "배우들의 연기가 훌륭했어요",
    "I was bored througtout the film"
]
# 추론
results = classifier(texts)
for result in results:
    print(result)

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


{'label': '5 stars', 'score': 0.4925258755683899}
{'label': '1 star', 'score': 0.9602923393249512}
{'label': '5 stars', 'score': 0.49889132380485535}
{'label': '1 star', 'score': 0.4800568222999573}


한글에특화된 전이학습
  - 감성분석 : kobert
  - 텍스트 생성 : kogpt2

In [ ]:
# 분류를 하려면.. 모델을 분류모델로 전환
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)
# 한국어와 영어 혼합
texts = [
    "이 영화 정말 재미있어요",
    "배우들의 연기가 훌륭했어요",
]
# 추론
model.eval()
for text in texts:
  inputs = tokenizer(text, return_tensors='pt')
  outputs = model(**inputs)
  logits = outputs.logits
  predicted_class_id = logits.argmax().item()
  print(predicted_class_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1
0


In [ ]:
outputs.logits

tensor([[-0.2169, -0.1585]], grad_fn=<AddmmBackward0>)

In [ ]:
# kogpt2
model_name = 'skt/kogpt2-base-v2'
generator = pipeline('text-generation', model=model_name, tokenizer=model_name)
# 프롬프트
prompt = '오늘의 주요뉴스!'
# 추론
generated_text = generator(prompt, max_length=50, truncation=True,num_return_sequences=1)[0]['generated_text']
print(generated_text)

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Device set to use cpu


오늘의 주요뉴스!! 박성미 전 대표의 발언에 따르면, 김 후보는 이날 밤 10시쯤 서울 여의도 당사에서 최고위원들과 함께 기자회견을 열어 "새누리당은 국민 앞에서 공당의 면모를 보여주고 국민에게 품격


지금까지는 기존의 베이스 모델 또는 특화 모델을 가져왔음
  - ▶ 원하는 데이터로 fine-tuning을 하려면

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer,TrainingArguments
model = AutoModelForSequenceClassification.from_pretrained("monologg/kobert")
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
!pip install datasets
from datasets import load_dataset
dataset = load_dataset('nsmc')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The repository for nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [31]:
# 파인튜닝할 데이터
import pandas as pd
pd.DataFrame(dataset['train'])

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [32]:
# 토큰화 함수
def tokenizer_fn(sentence):
  return tokenizer(sentence['document'],padding='max_length',truncation=True,return_tensors='pt')

In [33]:
tokenized_datasets = dataset.map(tokenizer_fn,batched=True)
tokenized_datasets

Map:   0%|          | 0/150000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [35]:
# 허깅페이스의 트레이너는 labels를 사용 기존 label용어를 변경
tokenized_datasets = tokenized_datasets.rename_column('label','labels')
tokenized_datasets.set_format('torch',columns=['input_ids','attention_mask','labels'])

ValueError: Original column name label not in the dataset. Current columns in the dataset: ['id', 'document', 'labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [36]:
# 시간단출을 위해 샘플링dmf gkrh  select를 슬라이싱대신 사용해서 데이터셋 자료구조가 유지됨
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(1000))
eval_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

In [38]:
# 학습
traning_args = TrainingArguments(
    output_dir='./results',          # output directory
    eval_strategy='epoch',     # evaluation strategy to adopt during training
    learning_rate=2e-5,              # number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    num_train_epochs=3,              # total number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,               # log saving step
    save_strategy='epoch',           # save strategy
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy',
    report_to='none'                 #wandb 사용 안함
)
# 트레이너
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
  lables = pred.label_ids
  preds = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(lables, preds, average='binary')
  acc = accuracy_score(lables, preds)
  return {
      'accuracy': acc,
      'f1': f1,
      'precision': precision,
      'recall': recall
  }

trainer = Trainer(
    model=model,
    args=traning_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # processing_class=tokenizer,  # tokenizer 라는 arg 명은 앞으로 사용 안할 예정
    compute_metrics=compute_metrics,

)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.796537,0.684000,0.728055,0.625740,0.870370
2,0.295400,0.658026,0.737000,0.753052,0.692573,0.825103
3,0.295400,0.692502,0.743000,0.740666,0.726733,0.755144


TrainOutput(global_step=189, training_loss=0.250275798575588, metrics={'train_runtime': 371.2584, 'train_samples_per_second': 8.081, 'train_steps_per_second': 0.509, 'total_flos': 789333166080000.0, 'train_loss': 0.250275798575588, 'epoch': 3.0})

In [40]:
# 필요시 저장
model.save_pretrained('kobert_finetuned')
tokenizer.save_vocabulary('kobert_finetuned')

('kobert_finetuned/tokenizer_78b3253a26.model', 'kobert_finetuned/vocab.txt')

In [41]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# 추론 함수 작성
import torch
def predict_sentiment(text,model,tokenizer):
  model.eval()
  inputs = tokenizer(text,return_tensors='pt',truncation=True,padding=True).to(device)
  with torch.no_grad():
    outputs = model(**inputs)
  logits = outputs.logits
  predicted_class = torch.argmax(logits,dim=1).item()
  return '긍정' if predicted_class == 1 else '부정'

In [42]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def predict_sentiment(text, model, tokenizer):
    model.to(device)
    model.eval()

    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return '긍정' if predicted_class == 1 else '부정'

In [43]:
predict_sentiment(  "시간이 아까울 정도로 별로였어요 ",model,tokenizer)

'부정'

Feature extraction

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [45]:
from transformers import AutoModel, AutoTokenizer
model_FE = AutoModel.from_pretrained("monologg/kobert")
tokenizer_FE = AutoTokenizer.from_pretrained("monologg/kobert", trust_remote_code=True)

In [46]:
import numpy as np
from tqdm import tqdm
import torch
# 디바이스 설정
model_FE.to(device)
model.eval() #평가모드(가중치 고정)
#임베딩 추출
dataset = load_dataset('nsmc')
def extract_embedding(dataset,tokenizer,model, max_samples=1000):
  embeddings, labels = [],[]
  for i, example in tqdm(enumerate(dataset)):
    if i >= max_samples:  # 샘플링
      break
    text = example['document']
    label = example['label']

    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
  # CLS 임베딩을 추출
    with torch.no_grad():
      outputs = model(**inputs)
      cls_embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    embeddings.append(cls_embedding)
    labels.append(label)
  return np.array(embeddings), np.array(labels)

In [47]:
# 훈련 및 임베딩 추출
train_embeddings, train_labels = extract_embedding(dataset['train'],tokenizer_FE,model_FE)
test_embeddings, test_labels = extract_embedding(dataset['test'],tokenizer_FE,model_FE)

1000it [00:09, 106.13it/s]
1000it [00:12, 77.96it/s]


In [48]:
train_embeddings.shape, train_labels.shape

((1000, 1, 768), (1000,))

In [49]:
# 분류기 학습
classifier = LogisticRegression(max_iter=1000)
classifier.fit(train_embeddings.squeeze(1), train_labels)

LogisticRegression(max_iter=1000)

In [50]:
test_predictions = classifier.predict(test_embeddings.squeeze(1))
print(classification_report(test_labels, test_predictions))

              precision    recall  f1-score   support

           0       0.68      0.70      0.69       492
           1       0.70      0.68      0.69       508

    accuracy                           0.69      1000
   macro avg       0.69      0.69      0.69      1000
weighted avg       0.69      0.69      0.69      1000



In [51]:
# 추론 함수
def predict_sentiment(text,tokenizer,model,classifier):
  model.eval()
  inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
  with torch.no_grad():
    outputs = model(**inputs)
    cls_embedding = outputs.last_hidden_state[:,0,:].cpu().numpy()  # cls 토큰 => 마지막 레이어의 토큰
    prediction = classifier.predict(cls_embedding)
    return "긍정" if prediction == 1 else '부정'

In [52]:
predict_sentiment('제작비라도 회수하면 다행일듯',tokenizer_FE,model_FE,classifier)

'부정'

FE에대한 개선방향
```
임베딩을 추출할때 batch 단위로 추출하면 더 빠르고 다양한 데이터 섞여서 좀더 학습에 유리
```

질의 응답(koBERT)
- 1. 원형 모델 그대로 사용
- 2. fine-turning
- 3. feature-extraction
- 주어진 문맥에서 질문에 대한 답변을 문맥내 특정 구간으로 추출
- 문맥 :  애플은 1976년에 스트브잡스와 스티브 워즈니악에 의해 설립되었다
- 질문 : 애플은 언제 설립되었나요?
- 답변 : 1976년
- 문맥의 질문을 입력으로 받아서 시작/끝 토큰 위치 예측
- koBERT는 [cls] 및 토큰 임베딩을 활용
- [CLS] 질문 [SEP] 문맥 [SEP]

In [11]:
# 1. base 모델 원형
# pipeline  AutoModel.... Answering
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "monologg/kobert"
model_QA_base = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer_QA_base = AutoTokenizer.from_pretrained(model_name)
# 파이프라인
qa_pipeline = pipeline('question-answering', model=model_QA_base, tokenizer=tokenizer_QA_base)
# 테스트데이터
context = "현대자동차는 다양한 자동차를 생산하고 있으며, 대표 차량으로는 쏘나타가 있다"
question = [
    '현대자동차는 어떤 차를 생산하나요?',
    '현대자동차의 대표 차량은?'
]
# 추론
results = qa_pipeline(context=context, question=question)
for result in results:
  print(result)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Device set to use cuda:0


{'score': 0.004318859428167343, 'start': 0, 'end': 6, 'answer': '현대자동차는'}
{'score': 0.004261780064553022, 'start': 0, 'end': 15, 'answer': '현대자동차는 다양한 자동차를'}


fine-turning

In [1]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [15]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
import torch
dataset = load_dataset('klue','mrc')
model_name = "monologg/kobert"
model_QA_FE = AutoModel.from_pretrained(model_name)
tokenizer_QA_FE = AutoTokenizer.from_pretrained(model_name)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_QA_FE.to(device)
model_QA_FE.eval()

The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(8002, 768, padding_idx=1)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [16]:
# 임베딩 추출함수
import torch
import numpy as np
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def extract_cls_embedding(dataset, tokenizer, model, max_samplings=1000):
  embeddings = []
  labels = []  # 1 답변가능, 0 답변 불가능
  for i ,example in tqdm(enumerate(dataset)):
    if i>= max_samplings:
      break
    question = example['question']
    context = example['context']
    answer = example['answers']['text'][0]
    label = 1 if answer else 0
    # 토큰화
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True,
                       padding='max_length', max_length=384).to(device)
    # cls 임베딩 추출
    with torch.no_grad():
      outputs = model(**inputs)
      cls_embedding = outputs.last_hidden_state[:,0,:].cpu().numpy()
      embeddings.append(cls_embedding[0])  # cls 임베딩
      labels.append(label)
  return np.array(embeddings), np.array(labels)

In [17]:
# 임베딩 추출
train_embeddings, train_labels = extract_cls_embedding(dataset['train'],tokenizer_QA_FE,model_QA_FE)
test_embeddings, test_labels = extract_cls_embedding(dataset['validation'],tokenizer_QA_FE,model_QA_FE
                                                     ,max_samplings=200)
# 머신러닝 학습
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression(max_iter=10)
model.fit(train_embeddings, train_labels)
model.score(train_embeddings, train_labels)  # accuracy

0it [00:00, ?it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
3it [00:00, 22.94it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the se

ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(1)

fine-turnig

In [27]:
!pip install datasets -q
from datasets import load_dataset

In [58]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = load_dataset("klue", "mrc")
model_name = 'klue/bert-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
def preprocess_function(examples):
    questions =  examples['question']
    contexts =  examples['context']
    answers =  examples['answers']
    tokenized = tokenizer(
        questions,
        contexts,
        max_length = 384,
        truncation = 'only_second',
        padding = 'max_length',
        return_offsets_mapping = True,
    )
    # 시작과 끝레이블 생성
    start_positions = []
    end_positions     = []
    for i in range(len(answers)):
      answer = answers[i]
      start_char = answer['answer_start'][0]
      end_char = start_char + len(answer['text'][0])
      offset_mapping = tokenized['offset_mapping'][i]
      token_start = None
      token_end = None
      for j, (start,end) in enumerate(offset_mapping):
        if start == start_char:
          token_start = j
        if end == end_char:
          token_end = j
          break
      start_positions.append(token_start if token_start is not None else 0)
      end_positions.append(token_end if token_end is not None else 0)
    tokenized['start_positions'] = start_positions
    tokenized['end_positions'] = end_positions
    return tokenized

# 데이터 전처리
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/5841 [00:00<?, ? examples/s]

In [59]:
train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(3000))
eval_dataset = tokenized_datasets['validation'].shuffle(seed=42).select(range(200))

In [60]:
# 평가 매트릭스
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  start_logits, end_logits = predictions
  start_positions, end_positions = labels

  start_pred = np.argmax(start_logits, axis=1)
  end_pred = np.argmax(end_logits, axis=1)

  exact_match = np.mean((start_pred == start_positions) & (end_pred == end_positions))
  return {'exact_match': exact_match}

In [61]:
# 학습설정
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir='./result', report_to="none",num_train_epochs=3)
trainer = Trainer(model=model, args=training_args
                  ,train_dataset=train_dataset
                  ,eval_dataset=eval_dataset
                  ,compute_metrics=compute_metrics
                  )
trainer.train()

Step,Training Loss
500,2.178400
1000,0.846100


TrainOutput(global_step=1125, training_loss=1.3954557766384548, metrics={'train_runtime': 709.5607, 'train_samples_per_second': 12.684, 'train_steps_per_second': 1.585, 'total_flos': 1763753107968000.0, 'train_loss': 1.3954557766384548, 'epoch': 3.0})

In [62]:
model.save_pretrained('./result')
tokenizer.save_pretrained('./result')

('./result/tokenizer_config.json',
 './result/special_tokens_map.json',
 './result/vocab.txt',
 './result/added_tokens.json',
 './result/tokenizer.json')

In [63]:
# 데이터 형태 확인
print(dataset['validation']['context'][0])
print(dataset['validation']['question'][0])
print(dataset['validation']['answers'][0]['text'])

BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.
말라카이트에서 나온 색깔을 사용한 에디션은?
['뉴 740Li 25주년 에디션', '뉴 740Li 25주년']


In [67]:
# 추론 함수
import torch
def predict_asnwer(context, question, model, tokenizer):
  model.eval()
  inputs = tokenizer(question, context, max_length=384
                     , truncation='only_second'
                     , padding='max_length'
                     , return_tensors='pt').to(device)
  with torch.no_grad():
    outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)
    # 답변 디코딩
    answer_tokens = inputs.input_ids[0][start_idx:end_idx+1]
    answer = tokenizer.decode(answer_tokens)
  return answer

In [65]:
context = "현대자동차는 다양한 자동차를 생산하고 있으며, 대표 차량으로는 쏘나타가 있다"
question = [
    '현대자동차는 어떤 차를 생산하나요?',
    '현대자동차의 대표 차량은?'
]
for q in question:
  answer = predict_asnwer(context, q, model, tokenizer)
  print(answer)

다양한 자동차를 생산하고 있으며, 대표 차량으로는 쏘나타
쏘나타


In [70]:
context = dataset['validation']['context'][0]
question = dataset['validation']['question'][0]
print(f'context : {context}')
print(f'question : {question}')
predict_asnwer(context, question, model, tokenizer)

context : BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.
question : 말라카이트에서 나온 색깔을 사용한 에디션은?


'뉴 740Li 25주년 에디션'